Web scraping com Python

Requerimentos

```
pip install beautifulsoup4
pip install scrapy
pip install Pillow
pip3 install requests bs4 tqdm
```

In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
import urllib.request
import shutil
import re # for regular expressions, can be removed if no test is used
from tqdm import tqdm # to print progress bar
from urllib.parse import urljoin, urlparse
import requests
import os 
#teste
#html = urlopen("http://mallet.pr.gov.br/Site_mallet/materias/2020/03_marco/18_covid_oficio/18_covid_oficio.asp")
#res = BeautifulSoup(html.read(),"html5lib");    
#print(res.title)

In [2]:
# Functions
def is_valid(url): #Checks whether `url` is a valid URL.
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)

def get_all_images(url): # Returns all image URLs on a single `url`
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    urls = []
    for img in tqdm(soup.find_all("img"), "Extracting images"):
        img_url = img.attrs.get("src")
        if not img_url:
            # if img does not contain src attribute, just skip
            continue
        # make the URL absolute by joining domain with the URL that is just extracted
        img_url = urljoin(url, img_url)
        # remove URLs like '/hsts-pixel.gif?c=3.2.5'
        try:
            pos = img_url.index("?")
            img_url = img_url[:pos]
        except ValueError:
            pass
        # finally, if the url is valid
        if is_valid(img_url):
            urls.append(img_url)
    return urls

In [3]:
def download(url, pathname): # Downloads a file given an URL and puts it in the folder `pathname`
    # if path doesn't exist, make that path dir
    if not os.path.isdir(pathname):
        os.makedirs(pathname)
    # download the body of response by chunk, not immediately
    response = requests.get(url, stream=True)
    # get the total file size
    file_size = int(response.headers.get("Content-Length", 0))
    # get the file name
    filename = os.path.join(pathname, url.split("/")[-1])
    # progress bar, changing the unit to bytes instead of iteration (default by tqdm)
    progress = tqdm(response.iter_content(1024), f"Downloading {filename}", total=file_size, unit="B", unit_scale=True, unit_divisor=1024)
    with open(filename, "wb") as f:
        for data in progress:
            # write data read to the file
            f.write(data)
            # update the progress bar manually
            progress.update(len(data))

            
def main(url, path):
    # get all images
    imgs = get_all_images(url)
    for img in imgs:
        # for each image, download it
        download(img, path)
        
main("http://mallet.pr.gov.br/Site_mallet/materias/2020/03_marco/18_covid_oficio/18_covid_oficio.asp", "covid19_mallet_scrappedImages")

Extracting images: 100%|██████████| 193/193 [00:00<00:00, 18088.19it/s]


In [4]:
#OCR
from pytesseract import Output
# importing modules
import cv2
from pytesseract import pytesseract
"""
from tesseract2dict import TessToDict
td=TessToDict()
"""
image = cv2.imread('/home/orfeu/python/covid19_mallet_scrappedImages/0309.jpg')
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
threshold_img = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
#cv2.imshow('threshold image', threshold_img) # display image
#cv2.waitKey(0) # Maintain output window until user presses a key
#cv2.destroyAllWindows() # Destroying present windows on screen

custom_config = r'-c preserve_interword_spaces=1 --oem 1 --psm 1' #configuring parameters for tesseract
# now feeding image to tesseract
details = pytesseract.image_to_data(threshold_img, output_type='data.frame', config=custom_config, lang='por')
#details = pytesseract.image_to_data(threshold_img, output_type='data.frame', config=custom_config, lang='por', pandas_config=None)

"""
word_dict=td.tess2dict(threshold_img,'out','outfolder')
text_plain=td.word2text(word_dict,(0,0,threshold_img.shape[1],threshold_img.shape[0]))
print(type(text_plain))
print(text_plain)
#print(type(word_dict))
#print(word_dict)
"""

#print(details.text)
dt = details.text # select columm
print(dt)

0                 NaN
1                 NaN
2                 NaN
3                 NaN
4                   .
5         CORONAVÍRUS
6                 NaN
7               CASOS
8                  EM
9              MALLET
10                NaN
11           03/09/20
12                NaN
13                NaN
14       Notificados:
15                790
16                NaN
17       Monitorados:
18                 14
19                NaN
20         Suspeitos:
21                 11
22                NaN
23         Positivos:
24                 52
25                NaN
26     Recuperados:47
27                NaN
28    Descartados:727
29                NaN
30            Óbitos:
31                  2
32                NaN
33                NaN
34                NaN
35               so..
36                  |
37         Secretaria
38                 da
39              Saúde
40                NaN
41       Informações:
42                  F
43          Mallet-PR
Name: text, dtype: object


In [8]:
import pandas as pd
#create empty dataframe
col_names = ['Notificados','Monitorados','Suspeitos','Positivos','Recuperados','Descartados','Obitos','Atuais','Data']
coviddf = pd.DataFrame(columns = col_names)

#seleciona itens 
des = dt[details["text"].str.contains("1|2|3|4|5|6|7|8|9|:",  na=False)]
print(des)

11           03/09/20
14       Notificados:
15                790
17       Monitorados:
18                 14
20         Suspeitos:
21                 11
23         Positivos:
24                 52
26     Recuperados:47
28    Descartados:727
30            Óbitos:
31                  2
41       Informações:
Name: text, dtype: object


In [6]:
#cria novo dict para conter novos dados
covidict = {"dDescartados": [], "dNotificados":[],"dMonitorados":[],"dSuspeitos":[],"dPositivos": [], "dRecuperados":[],"dObitos":[],"dAtuais":[] }
#covidict["dDescartados"].append({'Descartados':des.str.contains(":")})
covidict

{'dDescartados': [],
 'dNotificados': [],
 'dMonitorados': [],
 'dSuspeitos': [],
 'dPositivos': [],
 'dRecuperados': [],
 'dObitos': [],
 'dAtuais': []}

In [7]:
for index, i in enumerate(des):
    if i == True:
        #i = i.str.split(':')
        #print(i)
        #create a dict
        print("i == True")
        #coviddf = coviddf.append({'Descartados':i.str.contains(":")})#append to new datafram
        
    else:
        #print(i.find("1"))
        if i.find("Desca") == 0:
            
                print("Ächei Desca")
                #Descarta = i
                if len(i) > 12:# certo é >
                    i2 = i.replace("Descartados:", "")
                    print(len(i))
                    print("i = ", i," e i2 =", i2)
                    covidict["dDescartados"].append(int(i2))
                else:
                    #continue aqui
                    #i2 = i[index+1]
                    print(i2, "code aqui ser for menor, see enumerate")
                    
                #covidict["dDescartados"].append({'Descartados':i.str.contains(":")})
                #coviddf.append({'Descartados':i2}, ignore_index=True)
                
        # problema so rodas os outros, ver multiplos casos
        #i = i.find("Monit")
        if i.find("Monit") == 0: # monitorados esta em outra linha
                #Monitora = i
                print("Achei Monit!! ", "i é igual a ",i)
                if len(i) > 12:# certo é >
                    i2 = i.replace("Monitorados:", "")
                    print(len(i), "len de monitirados")
                    print("i = ", i," e i2 =", i2)
                    covidict["dMonitorados"].append(int(i2))
                else:
                    #continue aqui
                    #i2 = i[index+1]
                    print("i2 = ",i2," -- i =  ", i, "code aqui ser for menor, see enumerate")
                    
                #coviddf.append({'Monitorados':i}, ignore_index=True)
        #coviddf = coviddf.append({'Monitorados':Descarta, "Descartados": Monitora}, ignore_index=True)
        #i = i.split(':').str[1]
        #print(i)
        
covidict        
#coviddf        

Achei Monit!!  i é igual a  Monitorados:


NameError: name 'i2' is not defined

In [238]:
print(coviddf)        
# 
"""        
coviddf = coviddf.append({'Descartados':dt.iloc[0],'Suspeitos':dt.iloc[0]}, ignore_index=True)#append to new dataframe
coviddf
  """

  Notificados   Monitorados Suspeitos Positivos Recuperados      Descartados  \
0         NaN  Monitorados:       NaN       NaN         NaN              NaN   
1         NaN           NaN       NaN       NaN         NaN  Descartados:727   

  Obitos Atuais Data  
0    NaN    NaN  NaN  
1    NaN    NaN  NaN  


"        \ncoviddf = coviddf.append({'Descartados':dt.iloc[0],'Suspeitos':dt.iloc[0]}, ignore_index=True)#append to new dataframe\ncoviddf\n  "

In [88]:
      
        #des = dt.str.contains(':')
#or i in dt:
    #dt[details["text"].astype(str).str.endswith(":")].copy()
    #print(i)
 #   if i == i.endswith(':') 
 #   print(i)
    #i.info()
    #if i.str.endswith(':')]
        #print(i, "code aqui")
    #else:
        #print(i)
#des.describe()
#casep = dt[details["text"].astype(str).str.endswith(':')]
#casep.describe()
#des
#des
#result
#des

2
1
1
1
1
1


14     True
17    False
20    False
23    False
30    False
41    False
Name: text, dtype: bool

In [98]:

"""
import re
re.findall('Suspeitos', text_plain)
"""

des = dt[details["text"].astype(str).str.contains('Descartados:')].copy()# get data from series
#if len(sus.str.len()) > 12:
des1 = des.str.split(':').str[1]
#else:
#    print(sus.str.len())# pula linha e le # implemente

sus = dt[details["text"].astype(str).str.contains('Suspeitos:')].copy()# get data from series
"""
for i in sus:
    print(dt.iloc[i + 1]value[0])
"""
#if len(sus.str.len()) > 10: # mude < ou >
    ##sus = sus1.str.split(':').str[1]
    #sus = sus.str.split(':').str[1]
#else:
    ##--->
    ##sus1= sus
    #print(sus.str.len())# pula linha e le # implemente

    
    
for i in range(dt.size):
     #print ('now', dt[dt.index[i]])
     #if i==0: continue
     print (dt[details["text"].astype(str).str.contains('Suspeitos:')].copy())
     #if i==dt.size-1: continue
     #print ('  last: ', dt[dt.index[i-1]])
     #print (dt[dt.index[i+1]].astype(str).str.contains('Suspeitos:') )   

#create empty dataframe
col_names = ['Notificados','Monitorados','Suspeitos','Positivos','Recuperados','Descartados','Obitos','Atuais','Data']
coviddf = pd.DataFrame(columns = col_names)
coviddf = coviddf.append({'Descartados':des1.iloc[0],'Suspeitos':sus.iloc[0]}, ignore_index=True)#append to new dataframe
coviddf
#sus2.info()
#sus2.describe()

20    Suspeitos:
Name: text, dtype: object
20    Suspeitos:
Name: text, dtype: object
20    Suspeitos:
Name: text, dtype: object
20    Suspeitos:
Name: text, dtype: object
20    Suspeitos:
Name: text, dtype: object
20    Suspeitos:
Name: text, dtype: object
20    Suspeitos:
Name: text, dtype: object
20    Suspeitos:
Name: text, dtype: object
20    Suspeitos:
Name: text, dtype: object
20    Suspeitos:
Name: text, dtype: object
20    Suspeitos:
Name: text, dtype: object
20    Suspeitos:
Name: text, dtype: object
20    Suspeitos:
Name: text, dtype: object
20    Suspeitos:
Name: text, dtype: object
20    Suspeitos:
Name: text, dtype: object
20    Suspeitos:
Name: text, dtype: object
20    Suspeitos:
Name: text, dtype: object
20    Suspeitos:
Name: text, dtype: object
20    Suspeitos:
Name: text, dtype: object
20    Suspeitos:
Name: text, dtype: object
20    Suspeitos:
Name: text, dtype: object
20    Suspeitos:
Name: text, dtype: object
20    Suspeitos:
Name: text, dtype: object
20    Suspe

,Notificados,Monitorados,Suspeitos,Positivos,Recuperados,Descartados,Obitos,Atuais,Data
0,NaN,NaN,Suspeitos:,NaN,NaN,727,NaN,NaN,NaN


In [60]:
s0 = pd.Series([1, 2, 3])
s1 = pd.Series([10, 20, 30])
s2 = pd.Series([100, 200, 300])
df = pd.DataFrame({'col 0': s0, 'col 1': s1, 'col 2': s2})
df.index = ['row 0', 'row 1', 'row 2']

>>> df

s0.map({1: 1, 2: 4, 3: 9})

# Use a lambda function
s0.map(lambda x: x*x)

# Use a regular function
def square(x):
    return x*x
s0.map(square)

SyntaxError: invalid syntax (<ipython-input-60-c011adc66a65>, line 7)